In [1]:
import autogen

In [2]:
from splunk_functions import splunk_query

In [3]:
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

In [4]:
ASSISTANT_SYSTEM_MESSAGE = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block), shell script (in a sh coding block), or Splunk spl for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.

Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply "TERMINATE" in the end when everything is done.
"""


USERAGENT_SYSTEM_MESSAGE = """
You are a helpful AI assistant.
Solve tasks using your coding and language skills.
"""

In [5]:
indices = """
_internal
_audit
"""

In [6]:
FUNCTIONS = [
    {
        "name": "splunk_query",
        "description": "Use this function to query Splunk spl. Input should be a fully formed spl query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            SPL query extracting info to answer the user's question.
                            Splunk has the following indexes:
                            {indices}
                            The query should be returned in plain text, not in JSON.
                            """,
                },
            #     "earliest_time": {
            #         "type": "string",
            #         "description": "The earliest time for the search (default last 24 hours). Input should be in SPL format, e.g. '-24h@h'"
            #     },
            #    "latest_time": {
            #         "type": "string",
            #         "description": "The latest time for the search (default now). Input should be in SPL format, e.g.'now'"
            #     } 
            },
            "required": ["query"],
        },
    }
]

In [7]:
# create an AssistantAgent named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    system_message=ASSISTANT_SYSTEM_MESSAGE,
    llm_config={
        "seed": 40,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0,  # temperature for sampling,
        "functions": FUNCTIONS
    },  # configuration for autogen's enhanced inference API which is compatible with OpenAI API
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,  # set to True or image name like "python:3" to use docker
    },
)

user_proxy.register_function(
    function_map={
        'splunk_query': splunk_query
    }
)

In [8]:
# the assistant receives a message from the user_proxy, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""Find the most common completed search sting made by admin today.""",
)


user_proxy (to assistant):

Find the most common completed search sting made by admin today.

--------------------------------------------------------------------------------
assistant (to user_proxy):

To find the most common completed search string made by admin today, we can use the Splunk SPL query. The query will look into the _audit index, filter events by the user "admin" and the action "search", and then count the occurrences of each search string. 

Let's execute the Splunk SPL query. 

```spl
index=_audit action=search user=admin earliest=@d latest=now 
| stats count by search 
| sort - count 
| head 1
```

This query does the following:
1. `index=_audit action=search user=admin earliest=@d latest=now`: This filters the events in the _audit index where the action is "search", the user is "admin", and the time is between the start of today (@d) and now.
2. `stats count by search`: This counts the number of occurrences of each search string.
3. `sort - count`: This sorts the re